# Cadre du TP

Dans OCaml il est possible de dessiner de telles figures assez simplement. Pour cela, on utilise le module `Graphics`.

On fournit ici des fonctions de plus haut niveau permettant de travailler plus simplement.

On commence par évaluer la cellule suivante qui permettra de travailler avec des graphismes dans le navigateur. Pour une utilisation sur vos machines, c'est `Graphics` qu'il faut utiliser. On trouvera la documentation de ce module ici : https://ocaml.github.io/graphics/graphics/Graphics/index.html


On utilisera ici uniquement quatre fonction

* `commence_dessin` qui prend deux entiers `w` et `h` et permet de dessiner dans une image de $w \times h$ pixels
* `affiche_dessin` qui va afficher le dessin en cours
* `ligne` telle que `ligne (x1,y2) (x2,y2)` trace une ligne noire entre les points de ces coordonnées entières
* `lignef` qui fait comme `ligne` mais avec des coordonnées flottantes

Le but du TP est de faire des dessins de fractales, on va donc avoir de toutes petites lignes par moment.

In [3]:
open Graphics_js
let canvas = Basthon.create_canvas ()

val canvas : 'a = <poly>

In [5]:
let commence_dessin w h =
    open_canvas canvas;
    resize_window w h

val commence_dessin : int -> int -> unit = <fun>

In [7]:
let affiche_dessin () =
    Basthon.display_canvas canvas

val affiche_dessin : unit -> unit = <fun>

In [9]:
let iof = int_of_float

val iof : float -> int = <fun>

In [12]:
let foi = float_of_int

val foi : int -> float = <fun>

In [14]:
let ligne (x1,y1) (x2,y2) =
    set_color black;
    moveto x1 y1;
    lineto x2 y2

val ligne : int * int -> int * int -> unit = <fun>

In [16]:
let lignef (x1,y1) (x2,y2) =
    set_color black;
    moveto (iof x1) (iof y1);
    lineto (iof x2) (iof y2)

val lignef : float * float -> float * float -> unit = <fun>

Pour faire un dessin, il faudra écrire des instructions impératives avec des `;` entre elles, comme ici :

In [18]:
commence_dessin 300 300;
ligne (50,50) (250,50);
ligne (250,50) (150,250);
ligne (150,250) (50,50);
affiche_dessin()

- : unit = ()

# Premiers dessins

**Question** Faire un dessin dans une fenêtre de 800x800 et
Tracez les lignes suivantes, données par leurs deux extrémités :
(200,200),(600,200)$
*  $(200,600),(600,600)$
* $(400,200),(400,600)$



In [21]:
commence_dessin 800 800;
ligne (200,200) (600,200);
ligne (200,600) (600,600);
ligne (400,200) (400,600);

Line 4, characters 27-28:
Error: Syntax error


- : unit = ()

**Question** Écrire une fonction `motif : int * int -> int * int` qui trace le dessin suivant (**Attention** on suppose qu'on a déjà appelé `commence_dessin`, on ne veut que le les lignes ici)

![](https://marc.de-falco.fr/poly/assets/pics/mp2i_tp4_motif.png)

où $P$ est le point dont
les coordonnées ont été données en entrée. Cette fonction renvoie alors
les coordonnées du point $P'$.

*Note* on pourra considérer que la longueur de chaque segment est de 20 pixels.

In [41]:
let motif (x,y) =
    ligne (x,y) (x+20,y);
    ligne (x+20,y) (x+20,y+20);
    ligne (x+20,y+20) (x+40,y+20);
    ligne (x+40,y+20) (x+40,y);
    (x+40,y)

val motif : int * int -> int * int = <fun>

In [29]:
commence_dessin 200 200;
ignore (motif (50,50)); (* ici on utilise ignore pour indiquer 
      qu'on n'utilise pas la valeur de retour de motif *)
affiche_dessin ()

- : unit = ()

In [45]:
let rec iter f n x = 
    if n=0 then x else (iter f (n-1) (f x))

val iter : ('a -> 'a) -> int -> 'a -> 'a = <fun>

In [47]:
assert( iter (fun x -> x+1) 4 0 = 4);
assert( iter (fun x -> x^x) 3 "a" = "aaaaaaaa" );
"Youpi"

- : string = "Youpi"

In [49]:
(* On doit pouvoir afficher une frise *)
commence_dessin 800 200;
ignore( iter motif 10 (50,50) );
affiche_dessin()

- : unit = ()

Dans la suite, on va essayer de changer d'approche pour faire des fractales : 

* créer un motif qui puisse être dessiné entre deux points
* remplacer dans le motif l'usage d'une ligne par le motif lui-même. Dans l'exemple précédent cela reviendrait à ne garder qu'un grand créneau mais que chaque segment soit lui-même un petit créneau.

# Dessin dans un repère

On va commencer par définir des fonctions pour travailler en géométrie du plan avec des coordonnées flottantes. On identifie ici point et vecteur par leur couple de coordonnées de type `float * float`.

**Question** Écrire une fonction `somme : float * float -> float * float -> float  * float` qui effectue la somme de deux vecteurs.

In [57]:
let somme (a,b) (c,d) =
    (a+.c),(b+.d)

val somme : float * float -> float * float -> float * float = <fun>

In [59]:
assert( somme (1., 2.) (-1., 3.) = (0., 5.) );
"Bravo"

- : string = "Bravo"

**Question**  Écrire une fonction ```smult : float -> float * float -> float  * float```
    qui calcule $\lambda \vec{u}$ quand on appelle `smult l u`.

In [61]:
let smult l (a,b) = (l*.a,l*.b)

val smult : float -> float * float -> float * float = <fun>

In [63]:
assert(smult 2. (1.,2.) = (2.,4.));
"OK"

- : string = "OK"

**Question** Écrire une fonction  `vecteur : float * float -> float * float -> float  * float`
    qui calcule les coordonnées de $\vec{AB}$ quand on appelle `vecteur a b`.

In [211]:
let vecteur (a:float*float) (b:float*float) = ( fun (x1,y1) (x2,y2) -> x2-.x1,y2-.y1 ) a b 

val vecteur : float * float -> float * float -> float * float = <fun>

In [209]:
assert(vecteur (1.,2.) (3.,1.) = (2.,-1.));
"Oui !"

- : string = "Oui !"

**Question** Écrire une fonction 
`normal : float * float -> float  * float`
qui renvoie le vecteur normal $\vec{v}$ à un vecteur $\vec{u}$ passé en entrée, de même norme
et tel que les deux vecteurs $\{\vec{u},\vec{v}\}$ forment un repère direct.

**Note** il suffit de jouer sur les coordonnées ici, faites un dessin, la fonction est très simple.

In [77]:
let normal (a,b) = (-.b,a)

val normal : 'a * float -> float * 'a = <fun>

In [78]:
assert (normal (1.0,1.0) = (-1.,1.));
assert (normal (1.0,0.) = (0.,1.));
"OK"

- : string = "OK"

 En déduire une fonction
```
repere_local : float * float -> float * float 
    -> float * float -> float * float
```
telle que l'appel à `repere_local (x1,y1) (x2,y2) (a,b)` renvoie le
    point $C = A + a \vec{AB} + b \vec{n}$ où $A = (x_1,y_1)$, $B= (x_2,y_2)$ et
    $\vec{n}$ est le vecteur déduit de $\vec{AB}$ comme dans la question
    précédente.

![](https://marc.de-falco.fr/poly/assets/pics/mp2i_tp4_repere.png)

Attention à ne pas oublier d'ajouter les coordonnées de $A$ !


In [172]:
let repere_local ma mb (a,b) =
    let vec_ab = vecteur ma mb in
    somme ma ( somme (smult a vec_ab) (smult b (normal vec_ab)) ) 

val repere_local :
  float * float -> float * float -> float * float -> float * float = <fun>

# Première fractale

**Question** Écrire une fonction
```
brisure : float * float -> float * float -> unit
```
telle que l'appel à ```\verb=brisure=```sur les points $A$ et $B$ trace :

![](https://marc.de-falco.fr/poly/assets/pics/mp2i_tp4_brisure.png)
   
Ici les deux nouveaux points ont pour coordonnées $(1/2,1/4)$ et
    $(1/2,-1/4)$ dans le repère local construit sur $\vec{AB}$.

In [173]:
let brisure a b = 
    let c1 = (repere_local a b (0.5,0.25)) in
    let c2 = (repere_local a b (0.5,-0.25)) in
    lignef a c1;
    lignef c1 c2;
    lignef c2 b;
    

val brisure : float * float -> float * float -> unit = <fun>

In [174]:
commence_dessin 200 200;
brisure (0.,0.) (50.,100.);
brisure (50.,200.) (200.,0.);
affiche_dessin ()

- : unit = ()

Normalement la fonction précédente utilise trois appels à `lignef`. Or,
cette fonction a le même type que `brisure`. On pourrait donc
logiquement remplacer ces appels par des appels récursifs à `brisure`
pour obtenir un tracé plus complexe.

**Question**    
A partir du code de la fonction précédente, écrire une fonction
```
fbrisure : (float * float -> float * float -> unit)
    -> float * float -> float * float -> unit
```
qui prend en premier argument une fonction du type de `lignef`. La
    fonction partielle `fbrisure lignef` devra se comporter comme
    `brisure`.

In [175]:
let fbrisure f a b =
        let c1 = (repere_local a b (0.5,0.25)) in
        let c2 = (repere_local a b (0.5,-0.25)) in
        f a c1;
        f c1 c2;
        f c2 b;
        

val fbrisure :
  (float * float -> float * float -> 'a) ->
  float * float -> float * float -> 'a = <fun>

In [178]:
commence_dessin 200 200;
fbrisure lignef (0.,0.) (50.,100.);
fbrisure brisure (50.,200.) (200.,0.);
affiche_dessin ()

- : unit = ()

**Question**     Faites un tracé entre $(200.0,400.0)$ et $(600.0,400.0)$ pour les fonctions
    suivantes :
```
lignef
fbrisure lignef
fbrisure (fbrisure lignef)
fbrisure (fbrisure (fbrisure lignef))
```

In [180]:
commence_dessin 600 600;
lignef (200.0,400.0) (600.0,400.0);
fbrisure lignef (200.0,400.0) (600.0,400.0);
fbrisure (fbrisure lignef) (200.0,400.0) (600.0,400.0);
fbrisure (fbrisure (fbrisure lignef)) (200.0,400.0) (600.0,400.0);
affiche_dessin ()

- : unit = ()

 Écrire une fonction `iter_fbrisure` tel que les appels à
    `iter_fbrisure 0`, 
    `iter_fbrisure 1`, 
    `iter_fbrisure 2` et
    `iter_fbrisure 3` se ramènent aux fonctions de la question précédente.

In [183]:
let rec iter_fbrisure (n:int) : float*float -> float*float -> unit = 
    if n>0 then fbrisure (iter_fbrisure (n-1))
    else lignef

val iter_fbrisure : int -> float * float -> float * float -> unit = <fun>

In [184]:
commence_dessin 500 500;
iter_fbrisure 10 (50.,50.) (450.,450.);
affiche_dessin ()

- : unit = ()

# La machine à fractales

On va essayer ici d'adapter ce qu'on a fait pour ne pas travailler uniquement sur le motif `brisure` mais sur n'importe quel motif donné par la liste des points intermédiaires qui le constituent.

**Question** Écrire une fonction récursive
```
polyligne : (float * float) list -> unit
```
qui lorsqu'on appelle `polyligne [P1;P2;..;Pn]` trace les segments
$[P_1;P_2]$, $[P_2;P_3], \dots, [P_{n-1};P_n]$.

In [136]:
let rec polyligne l = 
    match l with 
    | []
    | [_] -> ()
    | a::b::q -> lignef a b ; polyligne (b::q)

val polyligne : (float * float) list -> unit = <fun>

In [185]:
(* doit afficher un carré ! *)
commence_dessin 300 300;
polyligne [ (50.,50.); (250.,50.); (250.,250.); (50.,250.); (50.,50.) ];
affiche_dessin ()

- : unit = ()

**Question** Écrire une fonction récursive 
```
polylocal : (float * float) list -> float * float -> 
    float * float ->  -> unit
```
qui lorsqu'on appelle `polylocal l a b` fonctionne comme `polyligne` mais
avec les coordonnées locales dans le repere construit sur $\vec{AB}$ pour les
points de `l`. On doit alors avoir 
```
polylocal [(0.0,0.0);(0.5,0.25);(0.5,-0.25);(1.0,0.0)] a b
```
qui se comporte exactement comme brisure.

**Note** on pourra utiliser un `List.map` pour se ramener à `polyligne` ou recoder la fonction précédente.

In [154]:
let rec polylocal l a b =
    polyligne (List.map (repere_local a b) l)

val polylocal :
  (float * float) list -> float * float -> float * float -> unit = <fun>

In [186]:
commence_dessin 300 300;
polylocal [(0.0,0.0);(0.5,0.25);(0.5,-0.25);(1.0,0.0)] (50.,50.) (250.,250.);
affiche_dessin ()

- : unit = ()

**Question** on va alors en déduire une fonction `fpolylocal` qui se comporte comme la précédente mais en appelant une fonction `f` passée en paramètre à la place de `lignef` :

In [187]:
let rec fpolylocal f l a b = 
    let rec polyf l f= 
        match l with 
        | []
        | [_] -> ();
        | a::b::q -> f a b ; polyf (b::q) f in
    polyf (List.map (repere_local a b) l) f

val fpolylocal :
  (float * float -> float * float -> 'a) ->
  (float * float) list -> float * float -> float * float -> unit = <fun>

In [188]:
commence_dessin 300 300;
let l = [(0.0,0.0);(0.5,0.25);(0.5,-0.25);(1.0,0.0)] in
fpolylocal (fpolylocal lignef l) l (50.,50.) (250.,250.);
affiche_dessin ()

- : unit = ()

**Question** Ecrire une fonction `fractale` qui adapte `iter_fbrisure` dans le
cas de la fonction précédente (`fpolylocal`). C'est le moment de contempler votre travail
avec les coefficients suivants, par exemple, mais surtout avec ceux qui
vous passent par la tete !

* $(0,0),(0.25,0.5),(0.5,0.0),(1.0,0)$.
* $(0,0),(0.33,0),(0.5,0.4),(0.66,0),(1.0,0)$.

In [197]:
let rec fractale (n:int) (l:(float * float) list): float * float -> float * float -> unit =
    if n>0 then fpolylocal (fractale (n-1) l) l 
    else lignef
    

    

val fractale :
  int -> (float * float) list -> float * float -> float * float -> unit =
  <fun>

In [198]:
commence_dessin 500 500;
fractale 10 [(0.,0.);(0.25,0.5);(0.5,0.);(1.,0.)] (100.,250.) (450.,250.);
affiche_dessin ()

- : unit = ()

In [199]:
commence_dessin 500 500;
fractale 6 [(0.,0.);(0.33,0.);(0.5,0.4);(0.66,0.);(1.,0.)] (0.,0.) (500.,0.);
affiche_dessin ()

- : unit = ()

In [222]:
commence_dessin 500 500;
fractale 5 [(0.,0.0);(0.33,0.);(0.5,0.4);(0.66,0.);(1.,0.)] (0.,0.) (500.,0.);
affiche_dessin ()

- : unit = ()